# Item 14: Sort by Complex Criteria Using the `key` Parameter

In [1]:
# Python's list built-in type provides a sort() method for ordering items in a list. By default, it orders the
# items in ascendin order
numbers = [93, 86, 11, 68, 70]
numbers.sort()
print(numbers)

[11, 68, 70, 86, 93]


In [3]:
# What does sort() do with objects?
class Tool:
    def __init__(self, name, weight):
        self.name = name
        self.weight = weight

    def __repr__(self):
        return f'Tool({self.name!r}, {self.weight})'

tools = [
    Tool('level', 3.5),
    Tool('hammer', 1.25),
    Tool('screwdriver', 0.5),
    Tool('chisel', 0.25)
]

# Sorting objects of this type doesn't work because sort() tries to cal comparison special methods that aren't
# defined in the class
tools.sort()

TypeError: '<' not supported between instances of 'Tool' and 'Tool'

Often there's an attribute on the object that we'd like to use for sorting. To support this use case, the `sort` method accepts a `key` parameter that's expected to be a function. The `key` function is passed a single argument, which is an item from the list that is being sorted. The return value of the `key` function should be a comparable value to use in place of an item for sorting purposes.

In [4]:
# Here the function allows us to sort the list of Tools objects alphabetically by their names
print('Unsorted: ', repr(tools))
tools.sort(key=lambda x: x.name)
print('\nSorted: ', tools)

Unsorted:  [Tool('level', 3.5), Tool('hammer', 1.25), Tool('screwdriver', 0.5), Tool('chisel', 0.25)]

Sorted:  [Tool('chisel', 0.25), Tool('hammer', 1.25), Tool('level', 3.5), Tool('screwdriver', 0.5)]


In [5]:
# Here we sort by the weight parameter
tools.sort(key=lambda x: x.weight)
print('By weight:', tools)

By weight: [Tool('chisel', 0.25), Tool('screwdriver', 0.5), Tool('hammer', 1.25), Tool('level', 3.5)]


In [6]:
# When dealing with strings, we can even the key function to do transformations on the values before sorting
# Here we apply the lower() method to each item to ensure that they're alphabetically ordered (since in the 
# natural lexical ordering of strings, capital letters come before lowercase letters)
places = ['home', 'work', 'New York', 'Paris']
places.sort()
print('Case sensitive: ', places)
places.sort(key=lambda x: x.lower())
print('Case insensitive: ', places)

Case sensitive:  ['New York', 'Paris', 'home', 'work']
Case insensitive:  ['home', 'New York', 'Paris', 'work']


In [7]:
# Sometimes we may want to use multiple criteria for sorting
# Say that we have a list of power tools and I want to sort them first by weight and then by name. How can we
# accomplish this?
power_tools = [
    Tool('drill', 4),
    Tool('circular saw', 5),
    Tool('jackhammer', 40),
    Tool('sander', 4)
] 

In [10]:
# The simplest solution in Python to this question is to use tuples. Tuples are comparable by default and have
# a natural ordering
saw = (5, 'circular saw')
jackhammer = (40, 'jackhammer')
assert not (jackhammer < saw) # Matches expectations

In [11]:
# If the first position in the tuples being compared are equal, then the tuple comparison will move on to the 
# second position, and so on
drill = (4, 'drill')
sander = (4, 'sander')
assert drill[0] == sander[0] # Same weight
assert drill[1] < sander[1] # Alphabetically less
assert drill < sander # Thus, drill comes first

In [14]:
# We can take advantage of this tuple comparison behavior in order to sort the list of power tools first by
# weight and then by name
power_tools.sort(key=lambda x: (x.weight, x.name))
print(power_tools)

[Tool('drill', 4), Tool('sander', 4), Tool('circular saw', 5), Tool('jackhammer', 40)]


In [15]:
# One limitation that we have with the key function returning a tuple is that the direction of sorting for all
# criteria must be the same (either all in ascending or all in descending order)
power_tools.sort(key=lambda x: (x.weight, x.name), reverse=True)
print(power_tools) # Makes all criteria descending

[Tool('jackhammer', 40), Tool('circular saw', 5), Tool('sander', 4), Tool('drill', 4)]


In [16]:
# For numerical values its possible to mix sorting directions by using the unary operator in the key function
# This negates one of the values in the returned tuple, effectively reversing its sort order while leaving the 
# others intact
power_tools.sort(key=lambda x: (-x.weight, x.name))
print(power_tools)

[Tool('jackhammer', 40), Tool('circular saw', 5), Tool('drill', 4), Tool('sander', 4)]


In [17]:
# Unary operator is not available for all types tho
power_tools.sort(key=lambda x: (x.weight, -x.name))
print(power_tools)

TypeError: bad operand type for unary -: 'str'

## The `sort` method of the `list` type will preserve the order of the input list when the key function returns values that are equal to each other

In [18]:
power_tools.sort(key=lambda x: x.name) # Name ascending
power_tools.sort(key=lambda x: x.weight, reverse=True) # Weight descending
print(power_tools)

[Tool('jackhammer', 40), Tool('circular saw', 5), Tool('drill', 4), Tool('sander', 4)]


In [20]:
# To understand why the code above works
# First call to sort puts the names in alphabetical order
power_tools.sort(key=lambda x: x.name) # Name ascending
print(power_tools)

[Tool('circular saw', 5), Tool('drill', 4), Tool('jackhammer', 40), Tool('sander', 4)]


In [21]:
# When the second sort call by weight descending is made, it sees that both 'sander' and 'drill' have a weight
# of 4. This causes the sort method toput boht items into the final result list in the same order that they 
# appeared in the original list, thus preserving their relative ordering by name ascending
power_tools.sort(key=lambda x: x.weight, reverse=True) # Weight descending
print(power_tools)

[Tool('jackhammer', 40), Tool('circular saw', 5), Tool('drill', 4), Tool('sander', 4)]
